# GP Regression on Molecules #

An example notebook for basic GP regression on a molecular dataset using a Tanimoto fingerprint kernel and the Photoswitch Dataset:

Paper: https://arxiv.org/abs/2008.03226

Code: https://github.com/Ryan-Rhys/The-Photoswitch-Dataset



In [ ]:
# Imports

# To import from the gprotorch package
import sys
sys.path.append('..')

from botorch import fit_gpytorch_model
import gpytorch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import torch

from gprotorch.dataloader import DataLoaderMP
from gprotorch.dataloader.data_utils import transform_data

We define our model. See

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html

for further examples!


In [ ]:
# We define our GP model using the Tanimoto kernel

from gprotorch.kernels.fingerprint_kernels.tanimoto_kernel import TanimotoKernel

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # We use the Tanimoto kernel to work with molecular fingerprint representations
        self.covar_module = gpytorch.kernels.ScaleKernel(TanimotoKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

We define our experiment parameters. In this case we are reproducing the results of the E isomer transition wavelength prediction task from https://arxiv.org/abs/2008.03226 using 20 random splits in the ratio 80/20.

In [ ]:
# Regression experiments parameters, number of random splits and split size

n_trials = 20
test_set_size = 0.2

Load the Photoswitch Dataset via the DataLoaderMP class which contains several molecular property prediction benchmark datasets!

In [ ]:
# Load the Photoswitch dataset

loader = DataLoaderMP()
loader.load_benchmark("Photoswitch", "../data/property_prediction/photoswitches.csv")

# Featurise the molecules. 
# We use the fragprints representations (a concatenation of Morgan fingerprints and RDKit fragment features)

loader.featurize('fragprints')
X = loader.features
y = loader.labels

# initialise performance metric lists

r2_list = []
rmse_list = []
mae_list = []

The training/evaluation loop

In [ ]:
for i in range(0, n_trials):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=i)

    #  We standardise the outputs but leave the inputs unchanged
    _, y_train, _, y_test, y_scaler = transform_data(X_train, y_train, X_test, y_test)

    # Convert numpy arrays to PyTorch tensors and flatten the label vectors
    X_train = torch.tensor(X_train.astype(np.float64))
    X_test = torch.tensor(X_test.astype(np.float64))
    y_train = torch.tensor(y_train).flatten()
    y_test = torch.tensor(y_test).flatten()
#     y_train = torch.tensor(y_train)
#     y_test = torch.tensor(y_test)

    # initialise GP likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(X_train, y_train, likelihood)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Use the BoTorch utility for fitting GPs in order to use the LBFGS-B optimiser (recommended)
    fit_gpytorch_model(mll)

    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    # mean and variance GP prediction
    f_pred = model(X_test)

    y_pred = f_pred.mean
    y_var = f_pred.variance

    # Transform back to real data space to compute metrics and detach gradients. Must unsqueeze dimension
    # to make compatible with inverse_transform in scikit-learn version > 1
    y_pred = y_scaler.inverse_transform(y_pred.detach().unsqueeze(dim=1))
    y_test = y_scaler.inverse_transform(y_test.detach().unsqueeze(dim=1))

    # Output Standardised RMSE and RMSE on Train Set
    y_train = y_train.detach()
    y_pred_train = model(X_train).mean.detach()
    train_rmse_stan = np.sqrt(mean_squared_error(y_train, y_pred_train))
    train_rmse = np.sqrt(mean_squared_error(y_scaler.inverse_transform(y_train.unsqueeze(dim=1)), y_scaler.inverse_transform(y_pred_train.unsqueeze(dim=1))))
    print("\nStandardised Train RMSE: {:.3f}".format(train_rmse_stan))
    print("Train RMSE: {:.3f}".format(train_rmse))

    # Compute R^2, RMSE and MAE on Test set
    score = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    print("\nR^2: {:.3f}".format(score))
    print("RMSE: {:.3f}".format(rmse))
    print("MAE: {:.3f}".format(mae))

    r2_list.append(score)
    rmse_list.append(rmse)
    mae_list.append(mae)

r2_list = np.array(r2_list)
rmse_list = np.array(rmse_list)
mae_list = np.array(mae_list)

print("\nmean R^2: {:.4f} +- {:.4f}".format(np.mean(r2_list), np.std(r2_list)/np.sqrt(len(r2_list))))
print("mean RMSE: {:.4f} +- {:.4f}".format(np.mean(rmse_list), np.std(rmse_list)/np.sqrt(len(rmse_list))))
print("mean MAE: {:.4f} +- {:.4f}\n".format(np.mean(mae_list), np.std(mae_list)/np.sqrt(len(mae_list))))

The mean RMSE should be ca. 20.9 +- 0.7 nanometres!